# Faster YC<sub>b</sub>C<sub>r</sub> Decoding

As discussed in the forums, most methods of loading a JPEG image output an RGB array. Converting this back to YC<sub>b</sub>C<sub>r</sub> can result in tiny errors which could be significant since the stenographic techniques used here rely on tiny changes in DCT space.

Rémi Cogranne has kindly provided us a method to decode from JPEG directly to YCbCr using `jpegio`, skipping the RGB step. https://www.kaggle.com/remicogranne/jpeg-explanations

This method can be a bit slow as discussed [here](https://www.kaggle.com/c/alaska2-image-steganalysis/discussion/150359#845590).

Let's use vectorised operations in NumPy to speed this up

In [ ]:
!pip install git+https://github.com/dwgoon/jpegio
import jpegio as jio

In [ ]:
import numpy as np
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt

INPUT_PATH = Path('/kaggle/input/alaska2-image-steganalysis/')

In [ ]:
files_cover = [x for x in (INPUT_PATH / 'Cover').glob('**/*') if x.is_file()]
files_cover.sort()

len(files_cover)

# The original version

In [ ]:
def JPEGdecompressYCbCr(path):
    jpegStruct = jio.read(str(path))

    nb_colors = len(jpegStruct.coef_arrays)

    [Col, Row] = np.meshgrid(range(8), range(8))
    T = 0.5 * np.cos(np.pi * (2 * Col + 1) * Row / (2 * 8))
    T[0, :] = T[0, :] / np.sqrt(2)

    sz = np.array(jpegStruct.coef_arrays[0].shape)

    imDecompressYCbCr = np.zeros([sz[0], sz[1], nb_colors])
    szDct = (sz / 8).astype("int")

    for ColorChannel in range(nb_colors):
        tmpPixels = np.zeros(sz)

        DCTcoefs = jpegStruct.coef_arrays[ColorChannel]
        if ColorChannel == 0:
            QM = jpegStruct.quant_tables[ColorChannel]
        else:
            QM = jpegStruct.quant_tables[1]

        for idxRow in range(szDct[0]):
            for idxCol in range(szDct[1]):
                D = DCTcoefs[
                    idxRow * 8 : (idxRow + 1) * 8, idxCol * 8 : (idxCol + 1) * 8
                ]
                tmpPixels[
                    idxRow * 8 : (idxRow + 1) * 8, idxCol * 8 : (idxCol + 1) * 8
                ] = np.dot(np.transpose(T), np.dot(QM * D, T))
        imDecompressYCbCr[:, :, ColorChannel] = tmpPixels
        
    return imDecompressYCbCr.astype(np.float32)

In [ ]:
img_v1 = JPEGdecompressYCbCr(files_cover[0])
img_v1.shape

# Vectorised version
The nested `for` loops are causing a large slowdown. We can try two methods of speeding this up:
* Using `np.einsum`
* Using matrix multiplication

In [ ]:
def JPEGdecompressYCbCr_v2(path):
    jpegStruct = jio.read(str(path))

    [col, row] = np.meshgrid(range(8), range(8))
    T = 0.5 * np.cos(np.pi * (2 * col + 1) * row / (2 * 8))
    T[0, :] = T[0, :] / np.sqrt(2)

    img_dims = np.array(jpegStruct.coef_arrays[0].shape)
    n_blocks = img_dims // 8
    broadcast_dims = (n_blocks[0], 8, n_blocks[1], 8)
    
    YCbCr = []
    for i, dct_coeffs, in enumerate(jpegStruct.coef_arrays):

        if i == 0:
            QM = jpegStruct.quant_tables[i]
        else:
            QM = jpegStruct.quant_tables[1]
        
        t = np.broadcast_to(T.reshape(1, 8, 1, 8), broadcast_dims)
        qm = np.broadcast_to(QM.reshape(1, 8, 1, 8), broadcast_dims)
        dct_coeffs = dct_coeffs.reshape(broadcast_dims)
        
        a = np.einsum('ijkl,ilkm->ijkm', qm * dct_coeffs, t)
        b = np.einsum('ijkl,ilkm->ijkm', np.transpose(t, axes=(0, 3, 2, 1)), a)
        YCbCr.append(b.reshape(img_dims))
                    
    return np.stack(YCbCr, -1).astype(np.float32)


def JPEGdecompressYCbCr_v3(path):
    jpegStruct = jio.read(str(path))

    [col, row] = np.meshgrid(range(8), range(8))
    T = 0.5 * np.cos(np.pi * (2 * col + 1) * row / (2 * 8))
    T[0, :] = T[0, :] / np.sqrt(2)

    img_dims = np.array(jpegStruct.coef_arrays[0].shape)
    n_blocks = img_dims // 8
    broadcast_dims = (n_blocks[0], 8, n_blocks[1], 8)
    
    YCbCr = []
    for i, dct_coeffs, in enumerate(jpegStruct.coef_arrays):

        if i == 0:
            QM = jpegStruct.quant_tables[i]
        else:
            QM = jpegStruct.quant_tables[1]
        
        t = np.broadcast_to(T.reshape(1, 8, 1, 8), broadcast_dims)
        qm = np.broadcast_to(QM.reshape(1, 8, 1, 8), broadcast_dims)
        dct_coeffs = dct_coeffs.reshape(broadcast_dims)
        
        a = np.transpose(t, axes=(0, 2, 3, 1))
        b = (qm * dct_coeffs).transpose(0,2,1,3)
        c = t.transpose(0,2,1,3)
                
        z = a @ b @ c
        z = z.transpose(0,2,1,3)
        YCbCr.append(z.reshape(img_dims))
                    
    return np.stack(YCbCr, -1).astype(np.float32)

In [ ]:
img_v2 = JPEGdecompressYCbCr_v2(files_cover[0])  # np.einsum version
img_v3 = JPEGdecompressYCbCr_v3(files_cover[0])  # matrix multiplication version
img_v2.shape, img_v3.shape

In [ ]:
# Check if they are the same
np.allclose(img_v1, img_v2), np.allclose(img_v1, img_v3)

# Speed comparison

In [ ]:
%%timeit -n 100
JPEGdecompressYCbCr(files_cover[0])

In [ ]:
%%timeit -n 100
JPEGdecompressYCbCr_v2(files_cover[0])

In [ ]:
%%timeit -n 100
JPEGdecompressYCbCr_v3(files_cover[0])

Nice! Quite a decent speedup. Looks like `JPEGdecompressYCbCr_v3` which uses raw matrix multiplication is faster than the Einstein summation method

Let's compare the speed to PIL just for fun

In [ ]:
%%timeit -n 100
Image.open(files_cover[0]).convert('YCbCr')

# Comparing to the PIL conversion
Note that to directly compare the PIL array we need to convert fron `uint8` to either a signed `int` or `float` and subtract 128 so that it is zero-centred

In [ ]:
img_pil = np.array(Image.open(files_cover[0]).convert('YCbCr'), dtype=np.int32) - 128
img_pil[0]

In [ ]:
img_v3[0]

In [ ]:
difference = img_pil - img_v3
print(f"Mean difference: {np.mean(difference):0.5f}, Std Dev: {np.std(difference):0.5f}")
plt.hist(difference.flatten(), bins=50);

Not a huge difference, but +/-1 is sometimes all that is needed to hide bits in stenography